In [1]:
import matplotlib
%matplotlib nbagg
from matplotlib import pyplot

In [2]:
from statiskit import (linalg,
                       core,
                       pgm)

In [3]:
import math
import os

# Simuler un graphe cordal $\Theta$ et les données data avec le fichier sampling

In [4]:
if not 'K' in os.environ:
    os.environ['K'] = str(10)
K = int(os.environ.get('K'))
simulation = !jupyter nbconvert --ExecutePreprocessor.timeout=3600 --to notebook --execute sampling.ipynb --output sampling.ipynb
graph = pgm.read_gml(os.path.join('results', 'graph.gml'))
while not graph.is_chordal:
    simulation = !jupyter nbconvert --ExecutePreprocessor.timeout=3600 --to notebook --execute sampling.ipynb --output sampling.ipynb
    graph = pgm.read_gml(os.path.join('results', 'graph.gml'))

data = core.read_csv(os.path.join('results', 'data.csv'))
data


,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9
0,$0.45$,$-0.37$,$-0.34$,$1.46$,$0.90$,$-0.22$,$1.61$,$0.59$,$-1.39$,$-0.32$
1,$0.10$,$-0.50$,$-0.59$,$-0.28$,$1.52$,$-1.74$,$2.19$,$-0.00$,$0.79$,$0.05$
2,$-0.80$,$-0.42$,$0.91$,$0.72$,$1.31$,$-1.42$,$2.92$,$0.84$,$0.88$,$-0.36$
3,$-0.61$,$-0.44$,$0.39$,$-0.89$,$1.10$,$-0.56$,$-1.20$,$0.50$,$-0.63$,$0.08$
4,$0.55$,$-0.05$,$0.53$,$0.94$,$2.04$,$-1.39$,$-0.42$,$-0.52$,$-2.06$,$0.32$
5,$0.73$,$-0.29$,$1.45$,$0.15$,$0.20$,$1.94$,$-2.66$,$-0.05$,$1.74$,$0.34$
6,$-0.64$,$-0.96$,$0.60$,$1.12$,$0.92$,$1.02$,$1.73$,$0.80$,$-0.36$,$0.46$
7,$1.05$,$-0.71$,$1.37$,$0.60$,$-0.06$,$-0.19$,$-0.86$,$0.15$,$-0.77$,$-0.82$
8,$2.55$,$-0.82$,$0.75$,$-0.94$,$0.03$,$-0.78$,$-1.40$,$0.22$,$0.15$,$1.09$
9,$0.16$,$-0.57$,$0.24$,$-0.47$,$-0.49$,$1.21$,$-0.16$,$0.47$,$-0.66$,$-0.58$


# Estimation par   newton-raphson

In [5]:
algo = os.environ.get('ALGO', 'nr')

In [6]:
import time
prev = time.time()
nre = pgm.graphical_gaussian_estimation(algo=algo,
                                        data=data,
                                        graph=graph)
curr = time.time()

# Estimation par maximum vraisemblance MLE

In [9]:
import sys 
try:
    mle = pgm.graphical_gaussian_estimation(algo='ml',
                                        data=data,
                                        graph=graph)
except:
    mle = nre
    print ("theta non semi-def positive")
#finally:
#    sys.exit(0) 

# Calcul de $\widehat{\Theta}_{ML}$  et  $\widehat{\Theta}_{NR}$

In [13]:
theta_ML = mle.estimated.graph
theta_NR = nre.estimated.graph

AttributeError: 'NREstimator' object has no attribute 'estimated'

In [ ]:
fig = pyplot.figure()
axes = graph.to_matrix().plot(axes = fig.add_subplot(131))
axes.set_title(r'$\Theta$')
axes = theta_ML.to_matrix().plot(axes = fig.add_subplot(132))
axes.set_title(r'$\widehat{\Theta}_{ML}$')
axes = theta_NR.to_matrix().plot(axes = fig.add_subplot(133))
axes.set_title(r'$\widehat{\Theta}_{NR}$')

# Comparaison des deux algorithmes.

In [ ]:

LLH_ML = 2 * mle.estimated.loglikelihood(data)

LLH_NR = 2 * nre.estimated.loglikelihood(data)


(LLH_ML - LLH_NR )/LLH_ML

In [ ]:
LLH_ML, LLH_NR

In [ ]:
import math
with open(os.path.join('results', 'estimation.csv'), 'a') as filehandler:
    filehandler.write(','.join([str(K), str(algo), str(curr-prev), str((LLH_ML - LLH_NR )/math.fabs(LLH_ML))]) + '\n')